In [1]:
import pandas as pd
import joblib 
import numpy as np 
import datetime as datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import re
from datetime import datetime


**Only cleaning `SEC 10K` data since `crowdfunding` data proved to not be useful for this project**

In [2]:
# crowdfunding = joblib.load("crowdfunding.pkl")
annual = joblib.load("sec_10k_raw.pkl")

In [3]:
unique_co = pd.DataFrame(annual['name'].value_counts()).reset_index().sort_values(by ='name',ascending=False)
unique_co

,index,name
0,DUKE ENERGY CORP,932
1,ENTERGY CORP /DE/,902
2,AMERICAN ELECTRIC POWER CO INC,797
3,EXELON CORP,758
4,SOUTHERN CO,726
...,...,...
2577,PAIN THERAPEUTICS INC,1
2578,"COLUMBIA FINANCIAL, INC.",1
2579,"STRATEGIC HOTELS & RESORTS, INC",1
2580,"ENERGY HOLDINGS INTERNATIONAL, INC.",1


In [4]:
def extract_ticker(df,col):
    df[col] = df[col].map(lambda x: x.split('-')[0])
    df[col] = df[col].map(lambda x: re.findall('\d*\D+',x)[0])
    df[col] = df[col].map(lambda x: x.upper())
        
    return df[col]
    

In [5]:
annual['instance'] = extract_ticker(annual,'instance')

In [6]:
annual['fp'].value_counts()

FY    114475
Name: fp, dtype: int64

In [7]:
annual.rename(columns={'instance':'ticker',
'ddate':'end_date',
'period':'balance_sheet_date',
'fy':'fiscal_year',
'filed':'filed_date',
'Assets':'assets',
'CashAndCashEquivalentsAtCarryingValue':'cash_and_cash_equivalent_sat_carrying_value',
'CommonStockSharesAuthorized':'common_stock_shares_authorized',
'EarningsPerShareBasic':'earnings_per_share_basic',
'EarningsPerShareDiluted':'earnings_per_share_diluted',
'LiabilitiesAndStockholdersEquity':'liabilities_and_stock_holders_equity',
'NetCashProvidedByUsedInFinancingActivities':'net_cash_provided_by_used_in_financing_activities',
'NetCashProvidedByUsedInOperatingActivities':'net_cash_provided_by_used_in_operating_activities',
'NetIncomeLoss':'net_income_loss',
'OperatingIncomeLoss':'operating_incomeloss',
'RetainedEarningsAccumulatedDeficit':'retained_earnings_accumulated_deficit',
'StockholdersEquity':'stockholders_equity'},inplace=True)

In [8]:
pd.set_option('display.max_columns',500)
annual

tag,version,coreg,end_date,qtrs,uom,adsh,cik,name,balance_sheet_date,fiscal_year,fp,filed_date,form,ticker,assets,cash_and_cash_equivalent_sat_carrying_value,common_stock_shares_authorized,earnings_per_share_basic,earnings_per_share_diluted,liabilities_and_stock_holders_equity,net_cash_provided_by_used_in_financing_activities,net_cash_provided_by_used_in_operating_activities,net_income_loss,operating_incomeloss,retained_earnings_accumulated_deficit,stockholders_equity
0,us-gaap/2009,AllOther,20081231,0,USD,0000092122-11-000013,92122,SOUTHERN CO,20101231.0,2010.0,FY,20110225,10-K,SO,1.407000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,us-gaap/2009,AllOther,20091231,0,USD,0000092122-11-000013,92122,SOUTHERN CO,20101231.0,2010.0,FY,20110225,10-K,SO,1.223000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,us-gaap/2009,AllOther,20101231,0,USD,0000092122-11-000013,92122,SOUTHERN CO,20101231.0,2010.0,FY,20110225,10-K,SO,1.279000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,us-gaap/2009,AllOtherSubsidaries,20081231,0,USD,0000950123-11-019900,6769,APACHE CORP,20101231.0,2010.0,FY,20110228,10-K,APA,NaN,1.037000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,us-gaap/2009,AllOtherSubsidaries,20081231,4,USD,0000950123-11-019900,6769,APACHE CORP,20101231.0,2010.0,FY,20110228,10-K,APA,NaN,NaN,NaN,NaN,NaN,NaN,220000000.0,5.485000e+09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114470,us-gaap/2020,Captive,20200930,0,USD,0001584509-20-000230,1584509,ARAMARK,20200930.0,2020.0,FY,20201124,10-K,CIK,NaN,9.210000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114471,us-gaap/2020,GilatSatelliteNetworksLtd,20200630,0,USD,0000023197-20-000127,23197,COMTECH TELECOMMUNICATIONS CORP /DE/,20200731.0,2020.0,FY,20200929,10-K,CMTL,NaN,5.960100e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114472,us-gaap/2020,GilatSatelliteNetworksLtd,20200630,2,USD,0000023197-20-000127,23197,COMTECH TELECOMMUNICATIONS CORP /DE/,20200731.0,2020.0,FY,20200929,10-K,CMTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14219000.0,NaN,NaN
114473,us-gaap/2020,ICEEDistributor,20200930,4,USD,0001437749-20-024447,785956,J&J SNACK FOODS CORP,20200930.0,2020.0,FY,20201125,10-K,JJSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3600000.0,NaN,NaN


**Data Formatting**

In [9]:
def date_format(df,col):
    df[col] = df[col].astype(str) 
    df[col]= df[col].map(lambda x: datetime.strptime(x, '%Y%m%d').strftime('%Y-%m-%d'))
    df[col] = pd.to_datetime(df[col], errors = 'coerce')
    return df[col]

def year_format(df,col):
    df[col] = df[col].astype(int)
    return df[col]
 

In [10]:
date_format(annual,'end_date')

0        2008-12-31
1        2009-12-31
2        2010-12-31
3        2008-12-31
4        2008-12-31
            ...    
114470   2020-09-30
114471   2020-06-30
114472   2020-06-30
114473   2020-09-30
114474   2019-12-31
Name: end_date, Length: 114475, dtype: datetime64[ns]

In [11]:
date_format(annual,'filed_date')

0        2011-02-25
1        2011-02-25
2        2011-02-25
3        2011-02-28
4        2011-02-28
            ...    
114470   2020-11-24
114471   2020-09-29
114472   2020-09-29
114473   2020-11-25
114474   2020-03-30
Name: filed_date, Length: 114475, dtype: datetime64[ns]

In [12]:
year_format(annual,'fiscal_year') 

0         2010
1         2010
2         2010
3         2010
4         2010
          ... 
114470    2020
114471    2020
114472    2020
114473    2020
114474    2019
Name: fiscal_year, Length: 114475, dtype: int64

In [13]:
annual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114475 entries, 0 to 114474
Data columns (total 26 columns):
 #   Column                                             Non-Null Count   Dtype         
---  ------                                             --------------   -----         
 0   version                                            114475 non-null  object        
 1   coreg                                              114475 non-null  object        
 2   end_date                                           114475 non-null  datetime64[ns]
 3   qtrs                                               114475 non-null  int64         
 4   uom                                                114475 non-null  object        
 5   adsh                                               114475 non-null  object        
 6   cik                                                114475 non-null  object        
 7   name                                               114475 non-null  object        
 8   bala

In [14]:
annual_test = annual.drop(annual.columns[np.r_[0,1,3,4,5,6,7,8,10,12,16:19,24]], axis=1)
annual_test                  

tag,end_date,fiscal_year,filed_date,ticker,assets,cash_and_cash_equivalent_sat_carrying_value,liabilities_and_stock_holders_equity,net_cash_provided_by_used_in_financing_activities,net_cash_provided_by_used_in_operating_activities,net_income_loss,operating_incomeloss,stockholders_equity
0,2008-12-31,2010,2011-02-25,SO,1.407000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-12-31,2010,2011-02-25,SO,1.223000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-12-31,2010,2011-02-25,SO,1.279000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-12-31,2010,2011-02-28,APA,NaN,1.037000e+09,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-12-31,2010,2011-02-28,APA,NaN,NaN,NaN,220000000.0,5.485000e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
114470,2020-09-30,2020,2020-11-24,CIK,NaN,9.210000e+07,NaN,NaN,NaN,NaN,NaN,NaN
114471,2020-06-30,2020,2020-09-29,CMTL,NaN,5.960100e+07,NaN,NaN,NaN,NaN,NaN,NaN
114472,2020-06-30,2020,2020-09-29,CMTL,NaN,NaN,NaN,NaN,NaN,NaN,-14219000.0,NaN
114473,2020-09-30,2020,2020-11-25,JJSF,NaN,NaN,NaN,NaN,NaN,NaN,3600000.0,NaN


In [15]:
joblib.dump(annual_test,"sec_10k.pkl")


['sec_10k.pkl']